# Deeper into symbolic computations


This week we will explore sympy in more detail.

* What sympy's capabilities are

* How we can use sympy to solve problems in:
 - Differential equations
 - Algebra
 - Calculus
 - Linear algebra
 
* How sympy structures mathematical objects

* * *

**This week's events**
 
* Quiz in Monday's lab.  

* Your second homework assignment available by Thursday's class.

## Sympy. . . again

We have seen [a few examples](../Week.3/Lecture.1.ipynb) of symbolic computation in Python:

 * Defining symbolic expressions, such as:
     - Polynomials
     - Algebraic expressions involving addition, multiplication, and standard mathematical functions like $\sin(x)$ and $e^x$.
     - Symbolic expressions for algebraic number types, such as $\frac{1}{\sqrt{2}+1}$
 * Differentiation and integration of symbolic functions
 
In these notes we will explore the sympy library a little further, getting an idea for what it can do, how sympy works, and what its limitations are.  

* * *

## Features of Sympy

The [features](http://www.sympy.org/en/features.html) of Sympy are vast and the scope of the library is changing rather quickly. It aims to be able to accomplish all forms of symbolic computation that *can* in principle be done by a computer. 

The qualifier in the above sentence is rather important. There are many basic algebraic tasks that are *non-computable*, in the sense that we have proofs that it is *impossible* to write a computer program that computes the answer to certain algebraic problems.  A closely related fact is that many differential equations do not have closed-form solutions, i.e. their solutions are not expressible in terms of [*elementary functions*](https://en.wikipedia.org/wiki/Elementary_function). 

These issues lead to certain unavoidable problems in symbolic computation.  For some tasks, Sympy has effective algorithms that give useful answers in a reliable manner.  But for other kinds of problems, Sympy will *try* to answer your query, but there are *no* estimates for how long it might take, *nor* how much system memory it will require to complete the task you have asked of it.  In effect sympy may or may not give you an answer to these kinds of requests. 
 
We will talk a little about both sorts of algorithms available in sympy. 

* * *

## Algebraic expressions

In [6]:
import sympy as sp

st = sp.sqrt(2) ## the square root of 2.

## Sympy has a variety of output formats
print(st)
sp.pprint(st)
sp.pprint(st, use_unicode=False) ## ASCII output
print(sp.latex(st), " this is LaTeX")

sqrt(2)
√2
  ___
\/ 2 
\sqrt{2}  this is LaTeX


In [8]:
import fractions as fr
print("\nNotice we can not use sqrt(2) in a Fraction.")

#fr.Fraction(st, 2)
## Fraction library not part of sympy!
sp.pprint(st/2)


Notice we can not use sqrt(2) in a Fraction.
√2
──
2 


With Sympy, checking equality can be more involved than one expects.

For example, lets check if sqrt(2)/2 is the same as 1/sqrt(2).

In [11]:
sp.pprint(st/2)
sp.pprint(1/st)

√2
──
2 
√2
──
2 


In [ ]:
x = sp.Symbol('x')
sp.pprint(sp.Eq(st/2,1/st))
sp.pprint(sp.Eq(sp.sin(x), sp.cos(x)) )

In [16]:
f = sp.sin(1)
f.evalf()

0.841470984807897

But how about the two expressions:
    
$$\frac{1}{1 + \sqrt{2}} \text{ and } \sqrt{2} - 1 $$
    

In [ ]:
## these are the same numbers!
sp.pprint(1/(1+st))
sp.pprint(st-1)
## but sympy does not see it!
sp.pprint(sp.Eq(1/(1+st), st-1))

In [20]:
sp.pprint(1/st == st/2)
sp.pprint(1/(1+st) == st-1)

## == after sympy stores the expression in its chosen internal format
##    the expressions are syntactically identical

## sp.Eq puts in a little more effort! 

True
False


The sympy.simplify call can be used to check equality of expressions such as this.

Let's see if simplification is 'strong enough' to verify equality.

In [24]:
blah = 1/(1+st) - (st-1)
sp.pprint( blah ) ## should be zero

sp.pprint(sp.simplify(blah))

        1       
-√2 + ────── + 1
      1 + √2    
0


When one defines objects like $\sqrt{2}/2$ in sympy, it observes that you are dividing two objects that are powers of $\sqrt{2}$ so it simplifies both rapidly to a power of $\sqrt{2}$.  

The double equals symbol has *limited* utility for comparing sympy objects.  So please **use with care.**  Sympy only recognises objects as being *equal* when they are stored (internally) in identical syntax.  

 * Some objects, like $\sqrt{2}/2$ reduce in syntax immediately to a 'minimal representative', in this case $1/\sqrt{2}$. 
 * Other objects have no canonical minimal representative (to Sympy's knowledge) so the syntax you choose to represent your object matters. 

You can get a hint as to how useful the double equals symbol is, by determining the data type sympy is using to store your object. 

In [33]:
print(type(st-1))
sp.pprint(st - 1)
sp.pprint(1/(st-1))

<class 'sympy.core.add.Add'>
-1 + √2
   1   
───────
-1 + √2


This is a *huge* **clue** as to how sympy thinks.  A sympy algebraic expression is stored internally in what is known as a **rooted tree**. 

<div style="display: inline-block; margin-right: 12px; margin-left: 50px">
<img src="pics/tree1.png" width="150" height="150" class="alignleft" title="sqrt(2)"/></div>
<div style="display: inline-block; margin-right: 12px; margin-left: 50px">
<img src="pics/tree2.png" width="250" height="250" class="alignleft" title="sqrt(2)-1"/></div>
<div style="display: inline-block; margin-right: 12px; margin-left: 50px">
<img src="pics/tree3.png" width="300" height="300" class="alignleft" title="1/(1+sqrt(2))"/></div>

The **final** operation in the expression is the **base** or **root** of the tree.  When one calls the double-equal operation for sympy expressions in Python, what Python does is it checks to see if the roots are identical.  If the roots of the trees are identical, it recursively moves *up* the tree, to check if all the sub-trees are identical.  

Sympy will perform the most *elementary* simplifications automatically, such as $\frac{\sqrt{2}}{2} = \frac{1}{\sqrt{2}}$. The term *elementary* should be taken to mean the simplifications that cost Python essentially no time or processing power. Sympy only attempts this for objects where a *canonical form* is known. 

Checking equality of more complicated expressions involves more computation, quite often because there is *no* canonical form. 

* * * 

### Calculus: derivatives and anti-derivatives

Sympy can compute derivatives of symbolic functions, using exactly the same tools we use: 
 * The chain rule 
 * The product rule
 * A table of derivaties of elementary functions, such as $x^n$, $e^x$ and $\sin(x)$. 
 

In [40]:
 ## 1) calculus, differentiation, integration, limits.
x = sp.Symbol('x')
x,y = sp.symbols('x y')

f = x**3 + sp.sin(x)
print(f)
print(type(f))

x**3 + sin(x)
<class 'sympy.core.add.Add'>


In [41]:
## We can ask sympy to give a more pleasant presentation via pprint
sp.pprint(f)

 3         
x  + sin(x)


## Check a function is a solution to a differential equation.

We will work with the logistic DE $f'=f(1-f)$ and a known solution $f(x) = \frac{1}{1+e^{-x}}$.

In [44]:
f = 1/(1+sp.exp(-x))
sp.pprint(f)

fp = sp.diff(f, x)
sp.pprint(fp)

   1   
───────
     -x
1 + ℯ  
    -x    
   ℯ      
──────────
         2
⎛     -x⎞ 
⎝1 + ℯ  ⎠ 


Let's check that the above is a solution to the logistic DE.

In [45]:
## compute f' and f(1-f)
RHS = f*(1-f)
sp.pprint(RHS)

       1   
1 - ───────
         -x
    1 + ℯ  
───────────
       -x  
  1 + ℯ    


In [47]:
sp.pprint(sp.Eq(fp, RHS))
feq = sp.Eq(fp, RHS)
sp.pprint(sp.simplify(feq))

                    1   
             1 - ───────
    -x                -x
   ℯ             1 + ℯ  
────────── = ───────────
         2          -x  
⎛     -x⎞      1 + ℯ    
⎝1 + ℯ  ⎠               
True


As we saw before, Sympy's "==" operator does not investigate our concern very carefully -- it is essentially telling us whether or not the expressions look syntactically the same.  We already know they do not! Let's try asking sympy to think a little harder about this.

In [48]:
sp.pprint(fp == RHS)
## the syntax of the left hand side and righty hand side are not the same. 

False


**sp.simplify** applied to the difference is zero, so the expressions are equal.

* * *

## Anti-derivatives

Sympy also has the capacity to compute anti-derivatives. This is perhaps surprising since there are many functions in mathematics that *do not* have anti-derivatives that are expressable in terms of *elementary functions*.   For example,

$$f(x) = e^{-x^2}$$

can not be expressed as a finite combination (sums, products, powers, quotients, composites) of polynomials or trig functions.  

In [54]:
f = x**5
sp.pprint(f)
af = sp.integrate(f,x)
sp.pprint(af)
sp.pprint(sp.integrate(f, (x, 0, 1)))

 5
x 
 6
x 
──
6 
1/6


We can typeset formulas in Sympy, as well.  The **Integral** command gives us an un-evaluated integral. 

In [56]:
af = sp.Integral(f, x)
sp.pprint(af)

⌠      
⎮  5   
⎮ x  dx
⌡      


And one can demand that a passive anti-derivative expression be evaluated using ''.doit()''

In [57]:
sp.pprint(af.doit())

 6
x 
──
6 


One does not need to give concrete instances of functions to sympy -- one can ask for general rules as well.  For example.

In [60]:
a,b,c = sp.symbols('a b c')
f = a*x**b + c
af = sp.integrate(f, x)
sp.pprint(sp.Eq(sp.Integral(f,x), af))
print(type(af))

                    ⎛⎧log(x)  for b = -1⎞      
⌠                   ⎜⎪                  ⎟      
⎮ ⎛   b    ⎞        ⎜⎪ b + 1            ⎟      
⎮ ⎝a⋅x  + c⎠ dx = a⋅⎜⎨x                 ⎟ + c⋅x
⌡                   ⎜⎪──────  otherwise ⎟      
                    ⎜⎪b + 1             ⎟      
                    ⎝⎩                  ⎠      
<class 'sympy.core.add.Add'>


Sympy is perfectly content giving *complicated* answers.  This is a feature of its underlying data type, trees.  Notice the type that $\int f$ is.

Let's parse this integral with the **sp.srepr(.)** command. 

In [61]:
sp.srepr(af)

"Add(Mul(Symbol('a'), Piecewise(ExprCondPair(log(Symbol('x')), Equality(Symbol('b'), Integer(-1))), ExprCondPair(Mul(Pow(Symbol('x'), Add(Symbol('b'), Integer(1))), Pow(Add(Symbol('b'), Integer(1)), Integer(-1))), S.true))), Mul(Symbol('c'), Symbol('x')))"

This is perhaps a mouthful to parse so let us begin with a simpler expression, the anti-derivative of $x^5$.

In [62]:
sp.srepr(sp.integrate(x**5, x))

"Mul(Rational(1, 6), Pow(Symbol('x'), Integer(6)))"

Thus sympy internally represents $x^6/6$ as the multiplication of:

 1) The rational number $1/6$ with

 2) The $6$-th power of $x$, $x^6$. 
 
 * * *
 
 Let's be mean and ask sympy about $\int e^{-x^2} dx$

In [65]:
f = sp.exp(-x**2)
sp.pprint(f)

sp.pprint(sp.integrate(f, x))

   2
 -x 
ℯ   
√π⋅erf(x)
─────────
    2    


In [66]:
sp.erf?

This is better than no information at all. sympy is telling us that (a rescaling) of this is called the $erf$ or <a href="http://docs.sympy.org/0.7.1/modules/mpmath/functions/expintegrals.html#erf">*error function*</a>.  Sympy can work with this function. 

Whenever you need to know what a sympy (or Python) object is simply type in **object?**, for example **sp.erf?**